# Cross-Language Word Embeddings

In class, we discussed how we can reduce the dimensionality of word representations from their original vector space to an embedding space on the order of a few hundred dimensions. Different modeling choices for word embeddings may be ultimately evaluated by the effectiveness of classification or retrieval models.

In this assignment, however, we will consider another common method of evaluating word embeddings: by judging the usefulness of pairwise distances between words in the embedding space.

Follow along with the examples in this notebook, and implement the sections of code flagged with **TODO**.

In [1]:
import gensim
import numpy as np
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

We'll start by downloading a plain-text version of the plays of William Shakespeare.

In [2]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/shakespeare_plays.txt
lines = [s.split() for s in open('shakespeare_plays.txt')]

--2022-12-13 20:06:45--  http://www.ccs.neu.edu/home/dasmith/courses/cs6200/shakespeare_plays.txt
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.ccs.neu.edu/home/dasmith/courses/cs6200/shakespeare_plays.txt [following]
--2022-12-13 20:06:46--  https://www.ccs.neu.edu/home/dasmith/courses/cs6200/shakespeare_plays.txt
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4746840 (4.5M) [text/plain]
Saving to: ‘shakespeare_plays.txt’

shakespeare_plays.t 100%[===================>]   4.53M  3.48MB/s    in 1.3s    

2022-12-13 20:06:48 (3.48 MB/s) - ‘shakespeare_plays.txt’ saved [4746840/4746840]



Then, we'll estimate a simple word2vec model on the Shakespeare texts.

In [3]:
model = Word2Vec(lines)

Even with such a small training set size, you can perform some standard analogy tasks we discussed in class.

In [4]:
model.wv.most_similar(positive=['king','woman'], negative=['man'])

[('queen', 0.7911697626113892),
 ('prince', 0.7724463939666748),
 ('duke', 0.7428550720214844),
 ('york', 0.7409319877624512),
 ('clarence', 0.7393580079078674),
 ('warwick', 0.709246039390564),
 ('gloucester', 0.7075702548027039),
 ('france', 0.679452121257782),
 ('margaret', 0.6726255416870117),
 ('princess', 0.6689456701278687)]

In other words, we want a vector close to `king` and `woman` but subtracting the dimensions that are important to `man`, i.e., `queen`. Other words are mostly noble titles important in Shakespeare's "history" plays.

For the rest of this assignment, we will focus on finding words with similar embeddings, both within and across languages. For example, what words are similar to the name of the title character of *Othello*?

In [5]:
model.wv.most_similar(positive=['othello'])
#model.wv.most_similar(positive=['brutus'])

[('desdemona', 0.967544674873352),
 ('iago', 0.9441386461257935),
 ('ham', 0.9032383561134338),
 ('cassio', 0.903009831905365),
 ('cressida', 0.9018601179122925),
 ('imogen', 0.8982028961181641),
 ('cleopatra', 0.8936619758605957),
 ('pisanio', 0.8936125040054321),
 ('rosalind', 0.8899244070053101),
 ('emilia', 0.8834909200668335)]

If you know the play, you might see some familiar names.

This search uses cosine similarity. In the default API, you should see the same similarity between the words `othello` and `desdemona` as in the search results above.

In [6]:
model.wv.similarity('othello', 'desdemona')

0.9675447

**TODO**: Your **first task**, therefore, is to implement your own cosine similarity function so that you can reuse it outside of the context of the gensim model object.

In [7]:
## TODO: Implement cosim
import numpy as np
from numpy.linalg import norm

def cosim(v1, v2):
  ## return cosine similarity between v1 and v2
  return np.dot(v1, v2) / (norm(v1) * norm(v2))

## This should give a result similar to model.wv.similarity:
cosim(model.wv['othello'], model.wv['desdemona'])

0.9675446

## Evaluation

We could collect a lot of human judgments about how similar pairs of words, or pairs of Shakespearean characters, are. Then we could compare different word-embedding models by their ability to replicate these human judgments.

If we extend our ambition to multiple languages, however, we can use a word translation task to evaluate word embeddings.

We will use a subset of [Facebook AI's FastText cross-language embeddings](https://fasttext.cc/docs/en/aligned-vectors.html) for several languages. Your task will be to compare English both to French, and to *one more language* from the following set: Arabic, German, Portuguese, Russian, Spanish, Vietnamese, and Chinese.

In [8]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.en.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.fr.vec

# TODO: uncomment at least one of these to work with another language
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.ar.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.de.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.pt.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.ru.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.es.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.vi.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.zh.vec

--2022-12-13 20:07:09--  http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.en.vec
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.en.vec [following]
--2022-12-13 20:07:10--  https://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.en.vec
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67681172 (65M)
Saving to: ‘30k.en.vec’

30k.en.vec          100%[===================>]  64.54M  12.9MB/s    in 6.1s    

2022-12-13 20:07:17 (10.6 MB/s) - ‘30k.en.vec’ saved [67681172/67681172]

--2022-12-13 20:07:17--  http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.fr.vec
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.

We'll start by loading the word vectors from their textual file format to a dictionary mapping words to numpy arrays.

In [9]:
def vecref(s):
  (word, srec) = s.split(' ', 1)
  return (word, np.fromstring(srec, sep=' '))

def ftvectors(fname):
  return { k:v for (k, v) in [vecref(s) for s in open(fname)] if len(v) > 1} 

envec = ftvectors('30k.en.vec')
frvec = ftvectors('30k.fr.vec')

# TODO: load vectors for one more language, such as zhvec (Chinese)
# arvec = ftvectors('30k.ar.vec')
# devec = ftvectors('30k.de.vec')
# ptvec = ftvectors('30k.pt.vec')
# ruvec = ftvectors('30k.ru.vec')
# esvec = ftvectors('30k.es.vec')
# vivec = ftvectors('30k.vi.vec')
zhvec = ftvectors('30k.zh.vec')

**TODO**: Your next task is to write a simple function that takes a vector and a dictionary of vectors and finds the most similar item in the dictionary. For this assignment, a linear scan through the dictionary using your `cosim` function from above is acceptible.

In [11]:
## TODO: implement this search function
def mostSimilar(vec, vecDict):
  ## Use your cosim function from above
  compare = 0
  for k,v in vecDict.items():
    cos_result = cosim(vec, v)
    if cos_result >= compare:
      compare = cos_result
      mostSimilar = k
      similarity = compare
  return (mostSimilar, similarity)

## some example searches
[mostSimilar(envec[e], frvec) for e in ['computer', 'germany', 'matrix', 'physics', 'yeast']]

[('informatique', 0.5023827767603765),
 ('allemagne', 0.593718413875964),
 ('matrice', 0.5088361302065517),
 ('physique', 0.4555543434796394),
 ('fermentation', 0.3504105196166514)]

Some matches make more sense than others. Note that `computer` most closely matches `informatique`, the French term for *computer science*. If you looked further down the list, you would see `ordinateur`, the term for *computer*. This is one weakness of a focus only on embeddings for word *types* independent of context.

To evalute cross-language embeddings more broadly, we'll look at a dataset of links between Wikipedia articles.

In [12]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/links.tab
links = [s.split() for s in open('links.tab')]

--2022-12-13 20:07:42--  http://www.ccs.neu.edu/home/dasmith/courses/cs6200/links.tab
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.ccs.neu.edu/home/dasmith/courses/cs6200/links.tab [following]
--2022-12-13 20:07:42--  https://www.ccs.neu.edu/home/dasmith/courses/cs6200/links.tab
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1408915 (1.3M)
Saving to: ‘links.tab’

links.tab           100%[===================>]   1.34M  1.22MB/s    in 1.1s    

2022-12-13 20:07:44 (1.22 MB/s) - ‘links.tab’ saved [1408915/1408915]



This `links` variable consists of triples of `(English term, language, term in that language)`. For example, here is the link between English `academy` and French `académie`:

In [13]:
links[302]

['academy', 'fr', 'académie']

We construct a test set for English-French rom the first 1000 links between those languages.

In [14]:
frtest = [x for x in links if x[1] == "fr"][0:1000]
frtest[0:10]

[['aalborg', 'fr', 'aalborg'],
 ['aarhus', 'fr', 'aarhus'],
 ['aba', 'fr', 'aba'],
 ['abad', 'fr', 'abad'],
 ['abandon', 'fr', 'abandon'],
 ['abbas', 'fr', 'abbas'],
 ['abbasid', 'fr', 'abbassides'],
 ['abbess', 'fr', 'abbesse'],
 ['abbey', 'fr', 'abbaye'],
 ['abbot', 'fr', 'abbé']]

**TODO**: Evaluate the English and French embeddings by computing the proportion of English Wikipedia articles whose corresponding French article in this test set `frtest` is also the closest word in embedding space. Skip English articles not covered by the word embedding dictionary. Since many articles, e.g., about named entities have the same title in English and French, compute the baseline accuracy achieved by simply echoing the English title as if it were French. Remember to iterate only over English Wikipedia articles, not the entire embedding dictionary.

In [15]:
## TODO: Compute English-French Wikipedia retrieval accuracy.
accuracy = 0
counter = 0
flag = 0
for ele in range(len(frtest)):
  if frtest[ele][0] in envec and mostSimilar(envec[frtest[ele][0]], frvec)[0] == frtest[ele][2]:
    counter = counter + 1
  elif frtest[ele][0] not in envec:
    flag = flag + 1
accuracy = counter / (1000 - flag)
print(accuracy)

baselineAccuracy = 0
counter = 0
for ele in range(len(frtest)):
  if frtest[ele][0] == frtest[ele][2]:
    counter = counter + 1
baselineAccuracy = counter / 1000
print(baselineAccuracy)

0.575
0.661


**TODO**: Compute accuracy and baseline (identity function) acccuracy for the first 1000 links between English and another language besides French. Although the baseline will be lower for languages not written in the Roman alphabet (i.e., Arabic or Chinese), there are still many articles in those languages with headwords written in Roman characters.

In [16]:
## TODO: Compute English-X Wikipedia retrieval accuracy.
zhtest = [x for x in links if x[1] == "zh"][0:1000]
accuracy = 0
counter = 0
flag = 0
for ele in range(len(zhtest)):
  if zhtest[ele][0] in zhvec and mostSimilar(envec[zhtest[ele][0]], zhvec)[0] == zhtest[ele][2]:
    counter = counter + 1
  elif zhtest[ele][0] not in zhvec:
    flag = flag + 1
accuracy = counter / (1000 - flag)
print(accuracy)

baselineAccuracy = 0
counter = 0
for ele in range(len(zhtest)):
  if zhtest[ele][0] == zhtest[ele][2]:
    counter = counter + 1
baselineAccuracy = counter / 1000
print(baselineAccuracy)

0.2465753424657534
0.067


**TODO**: Find the 10 nearest neighbors of each English term to compute "recall at 10" and "mean reciprocal rank at 10".

In [174]:
## TODO: Compute recall@10 and MRR@10 when retrieving 10 nearest neighbors in French and some other language.
import time

def most10Similar(vec, vecDict):
  compare = 0
  contain = []
  for k,v in vecDict.items():
    cos_result = cosim(vec, v)
    if cos_result >= compare:
      compare = cos_result
      mostSimilar = k
      similarity = compare
      contain.append((mostSimilar, similarity))
  return contain[-10:]

eng_fr_recall = []
eng_fr_mrr = []
eng_zh_recall = []
eng_zh_mrr = []

start = time.perf_counter()
for ele in range(len(frtest)):
  neighbors = most10Similar(envec[frtest[ele][0]], frvec)
  recall = 0
  for item in neighbors:
    if item[0] == frtest[ele][2]:
      recall = 0.1
  eng_fr_recall.append((frtest[ele][0], recall))
  mrr = 0
  for item in range(len(neighbors)):
    if neighbors[item][0] == frtest[ele][2]:
      mrr = 1 / (10 - item)
  eng_fr_mrr.append((frtest[ele][0], mrr))

for ele in range(len(zhtest)):
  neighbors = most10Similar(envec[zhtest[ele][0]], zhvec)
  recall = 0
  for item in neighbors:
    if item[0] == zhtest[ele][2]:
      recall = 0.1
  eng_zh_recall.append((zhtest[ele][0], recall))
  mrr = 0
  for item in range(len(neighbors)):
    if neighbors[item][0] == zhtest[ele][2]:
      mrr = 1 / (10 - item)
  eng_zh_mrr.append((zhtest[ele][0], mrr))

end = time.perf_counter()
time_1 = round(end - start)

In [ ]:
eng_fr_recall 
eng_fr_mrr 
eng_zh_recall 
eng_zh_mrr 

## [50 points Extra Credit] Speeding up Vector Search

The list of Wikipedia headwords is short enough that a linear scan through the non-English language embeddings takes some time but is feasible. In a production system, you could index the word embeddings using SimHash or some other locality sensitive hashing scheme, as we discussed for duplicate detection, to speed up this process.

A relatively easy way to get started with fast vector similarity search is to install Meta's `faiss` (Facebook AI Similarity Search) package and read [the tutorial](https://github.com/facebookresearch/faiss/wiki/Getting-started).

In [ ]:
# Outside of colab, you may need a different package manager.
!python -m pip install --upgrade pip
!apt install libomp-dev
!pip install faiss
import faiss
# Use this line instead if you have a GPU.
# !python -m pip install --upgrade faiss faiss-gpu

**TODO**: Create three vector indexes, for the FastText embeddings of English, French, and the other language you evaluated above. Use `faiss` to find the 10 nearest neighbors for the top 1000 Wikipedia headwords you evaluated for English-French and the English-X as above.

First, measure the _effectiveness_ of this approximate vector search approach. How does the R@10 and MRR@10 using `faiss` compare to the brute-force search you did above?

Second, measure the _efficiency_ of this approach. How long in seconds does finding nearest neighbors for 1000 headwords by brute force compare to using `faiss`? (For this exercise, don't worry about amortizing indexing costs.)

In [175]:
amount = 10
d = 300
index_en = faiss.IndexFlatL2(d)
index_fr = faiss.IndexFlatL2(d)
index_zh = faiss.IndexFlatL2(d)

en_set = []
fr_set = []
zh_set = []

fr_search = []
zh_search = []

for k,v in envec.items():
  en_set.append(v)
for k,v in frvec.items():
  fr_set.append(v)
  fr_search.append(k)
for k,v in zhvec.items():
  zh_set.append(v)
  zh_search.append(k)

en_set = np.array(en_set).astype('float32')
fr_set = np.array(fr_set).astype('float32')
zh_set = np.array(zh_set).astype('float32')

index_en.add(en_set)
index_fr.add(fr_set)
index_zh.add(zh_set)

In [176]:
eng_fr_recall_faiss = []
eng_fr_mrr_faiss = []
eng_zh_recall_faiss = []
eng_zh_mrr_faiss = []

start2 = time.perf_counter()
for ele in range(len(frtest)):
  D, I = index_fr.search(np.array([envec[frtest[ele][0]]]).astype('float32'), amount)
  recall = 0
  for item in I[0]:
    if fr_search[item] == frtest[ele][2]:
      recall = 0.1
  eng_fr_recall_faiss.append((frtest[ele][0], recall))
  mrr = 0
  for item in range(len(I)):
    if fr_search[I[0][item]] == frtest[ele][2]:
      mrr = 1 / (10 - item)
  eng_fr_mrr_faiss.append((frtest[ele][0], mrr))

for ele in range(len(zhtest)):
  D, I = index_fr.search(np.array([envec[zhtest[ele][0]]]).astype('float32'), amount)
  recall = 0
  for item in I[0]:
    if zh_search[item] == zhtest[ele][2]:
      recall = 0.1
  eng_zh_recall_faiss.append((zhtest[ele][0], recall))
  mrr = 0
  for item in range(len(I)):
    if zh_search[I[0][item]] == zhtest[ele][2]:
      mrr = 1 / (10 - item)
  eng_zh_mrr_faiss.append((zhtest[ele][0], mrr))

end2 = time.perf_counter()
time_2 = round(end2 - start2)

In [ ]:
eng_fr_recall_faiss 
eng_fr_mrr_faiss 
eng_zh_recall_faiss 
eng_zh_mrr_faiss

In [177]:
print("Without faiss: " + str(time_1) + " seconds")
print("Use faiss: " + str(time_2) + " seconds")

Without faiss: 719 seconds
Use faiss: 12 seconds
